# Statistical Analysis

Runs our statistics on pickled results

In [ ]:
from lib.stats import *
from itertools import product
import numpy as np
import scipy.stats as stats
import pandas as pd
from functools import reduce

%load_ext autoreload
%autoreload 2

In [2]:
def format_stats(stats) -> str:
    ranksum_res = stats['ranksum']
    bootstrap_res = stats['bootstrap']
    pval = ranksum_res.pvalue
    lo = bootstrap_res.confidence_interval.low
    hi = bootstrap_res.confidence_interval.high
    
    sig_mark = ""
    if stats["sig_better"]:
        sig_mark = "✷"
    
    res = f"{sig_mark}p = {pval:#.4g}; ci = ({lo:#.4g}, {hi:#.4g})" 
    
    if stats["invalidate"]:
        res = "-"
    
    return res

In [3]:
def format_summary(
    incl_std: bool = False,
    incl_nsamples: bool = False,
):
    
    def curried(summary):
        mean = summary['mean']
        std = summary['std']
        prefix = ""
        suffix = ""
        
        if 'invalidate' in summary and summary["invalidate"]:
            return "-"

        
        if 'best' in summary and summary['best'] == True:
            prefix += '!'
        if 'sig_better_baseline' in summary and summary['sig_better_baseline'] == True:
            suffix += '*'
        if 'sig_better_ablation' in summary and summary['sig_better_ablation'] == True:
            suffix += '†'
        
        if incl_nsamples:
            prefix = f"{summary['nsamples']} | " + prefix
            
        if incl_std:
            return f"{prefix}{mean:#.4f} ± {std:#.4f} {suffix}"
        if not incl_std:
            return f"{prefix}{mean:#.4f}{suffix}"
    
    return curried

## Reflex Pred

### Stat Tests

In [4]:
reflex_pairs = get_pairs(reflexSystems)
reflex_pairs

[(<S.GRU: 'GRU'>, <S.TRANS: 'Transformer'>),
 (<S.GRU: 'GRU'>, <S.JGRU: 'JambuGRU'>),
 (<S.GRU: 'GRU'>, <S.JTRANS: 'JambuTransformer'>),
 (<S.TRANS: 'Transformer'>, <S.GRU: 'GRU'>),
 (<S.TRANS: 'Transformer'>, <S.JGRU: 'JambuGRU'>),
 (<S.TRANS: 'Transformer'>, <S.JTRANS: 'JambuTransformer'>),
 (<S.JGRU: 'JambuGRU'>, <S.GRU: 'GRU'>),
 (<S.JGRU: 'JambuGRU'>, <S.TRANS: 'Transformer'>),
 (<S.JGRU: 'JambuGRU'>, <S.JTRANS: 'JambuTransformer'>),
 (<S.JTRANS: 'JambuTransformer'>, <S.GRU: 'GRU'>),
 (<S.JTRANS: 'JambuTransformer'>, <S.TRANS: 'Transformer'>),
 (<S.JTRANS: 'JambuTransformer'>, <S.JGRU: 'JambuGRU'>)]

In [5]:
reflex_stats_res: dict[D, dict[tuple[S, S], dict[M, dict]]] = {}
for d in allD:
    reflex_stats_res[d] = run_tests(T.REF, d, reflex_pairs)

### Statistical Test Tables

In [6]:
reflex_stats_dfs = {}
for d in allD:
    df = pd.DataFrame.from_dict(reflex_stats_res[d], orient='index')
    # df = df.reindex(columns=[M.ACC, M.TED, M.TER, M.FER, M.BCF])
    reflex_stats_dfs[d] = df

In [7]:
for d in allD:
    display(d)
    tmp_df = reflex_stats_dfs[d].copy()
    for m in allM:
        tmp_df[m] = tmp_df[m].apply(format_stats)
    display(tmp_df)

<D.WIKIHAN: 'chinese_wikihan2022'>

M.ACC  \
S.GRU    S.TRANS     p = 0.4196; ci = (-0.002942, 0.003796)   
         S.JGRU     ✷p = 3.151e-08; ci = (0.01731, 0.02246)   
         S.JTRANS     p = 1.000; ci = (-0.01487, -0.009546)   
S.TRANS  S.GRU       p = 0.5804; ci = (-0.003970, 0.003117)   
         S.JGRU     ✷p = 3.151e-08; ci = (0.01578, 0.02301)   
         S.JTRANS     p = 1.000; ci = (-0.01624, -0.008911)   
S.JGRU   S.GRU         p = 1.000; ci = (-0.02234, -0.01730)   
         S.TRANS       p = 1.000; ci = (-0.02293, -0.01576)   
         S.JTRANS      p = 1.000; ci = (-0.03485, -0.02914)   
S.JTRANS S.GRU     ✷p = 3.663e-08; ci = (0.009566, 0.01483)   
         S.TRANS   ✷p = 1.281e-07; ci = (0.008990, 0.01612)   
         S.JGRU     ✷p = 3.151e-08; ci = (0.02925, 0.03488)   

                                                        M.TED  \
S.GRU    S.TRANS     p = 0.009302; ci = (-0.01123, 0.0001080)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.03550, -0.02682)   
         S.JTRANS         p = 1.000; ci = (0.006006, 0.01656)   
S.TRANS  S.GRU         p = 0.9907; ci = (-0.0003443, 0.01154)   
         S.JGRU     ✷p = 4.256e-08; ci = (-0.03178, -0.01951)   
         S.JTRANS          p = 1.000; ci = (0.01048, 0.02363)   
S.JGRU   S.GRU             p = 1.000; ci = (0.02652, 0.03548)   
         S.TRANS           p = 1.000; ci = (0.01963, 0.03169)   
         S.JTRANS          p = 1.000; ci = (0.03719, 0.04815)   
S.JTRANS S.GRU     ✷p = 3.337e-06; ci = (-0.01646, -0.006315)   
         S.TRANS    ✷p = 1.437e-06; ci = (-0.02360, -0.01045)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.04824, -0.03715)   

                                                         M.TER  \
S.GRU    S.TRANS     p = 0.009302; ci = (-0.003312, 3.214e-05)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.01047, -0.007912)   
         S.JTRANS         p = 1.000; ci = (0.001773, 0.004885)   
S.TRANS  S.GRU         p = 0.9907; ci = (-0.0001010, 0.003406)   
         S.JGRU    ✷p = 4.256e-08; ci = (-0.009376, -0.005756)   
         S.JTRANS         p = 1.000; ci = (0.003092, 0.006970)   
S.JGRU   S.GRU             p = 1.000; ci = (0.007826, 0.01047)   
         S.TRANS          p = 1.000; ci = (0.005787, 0.009348)   
         S.JTRANS           p = 1.000; ci = (0.01097, 0.01420)   
S.JTRANS S.GRU     ✷p = 3.337e-06; ci = (-0.004854, -0.001861)   
         S.TRANS   ✷p = 1.437e-06; ci = (-0.006963, -0.003083)   
         S.JGRU      ✷p = 3.151e-08; ci = (-0.01423, -0.01096)   

                                                          M.FER  \
S.GRU    S.TRANS    ✷p = 0.002070; ci = (-0.001132, -0.0001048)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.003253, -0.002233)   
         S.JTRANS         p = 1.000; ci = (0.0004927, 0.001472)   
S.TRANS  S.GRU           p = 0.9979; ci = (0.0001028, 0.001139)   
         S.JGRU     ✷p = 1.031e-07; ci = (-0.002685, -0.001596)   
         S.JTRANS          p = 1.000; ci = (0.001074, 0.002141)   
S.JGRU   S.GRU             p = 1.000; ci = (0.002241, 0.003264)   
         S.TRANS           p = 1.000; ci = (0.001601, 0.002689)   
         S.JTRANS          p = 1.000; ci = (0.003201, 0.004271)   
S.JTRANS S.GRU     ✷p = 1.552e-05; ci = (-0.001487, -0.0005011)   
         S.TRANS    ✷p = 5.608e-07; ci = (-0.002148, -0.001073)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.004273, -0.003234)   

                                                       M.BCF  
S.GRU    S.TRANS    p = 0.02415; ci = (-0.0003197, 0.004723)  
         S.JGRU      ✷p = 3.151e-08; ci = (0.01128, 0.01515)  
         S.JTRANS     p = 1.000; ci = (-0.007304, -0.002741)  
S.TRANS  S.GRU       p = 0.9758; ci = (-0.004865, 0.0003593)  
         S.JGRU     ✷p = 4.256e-08; ci = (0.008254, 0.01379)  
         S.JTRANS      p = 1.000; ci = (-0.01023, -0.004421)  
S.JGRU   S.GRU          p = 1.000; ci = (-0.01517, -0.01119)  
         S.TRANS       p = 1.000; ci = (-0.01379, -0.008362)  
         S.JTRANS       p = 1.000; ci = (-0.02075, -0.01597)  
S.JTRANS S.GRU     ✷p = 2.7

<D.WIKIHAN_A: 'chinese_wikihan2022_augmented'>

M.ACC  \
S.GRU    S.TRANS       p = 1.000; ci = (-0.01199, -0.005297)   
         S.JGRU     ✷p = 1.102e-06; ci = (0.007426, 0.01664)   
         S.JTRANS       p = 1.000; ci = (-0.01577, -0.01014)   
S.TRANS  S.GRU      ✷p = 9.641e-07; ci = (0.005321, 0.01209)   
         S.JGRU      ✷p = 3.151e-08; ci = (0.01616, 0.02522)   
         S.JTRANS    p = 0.9999; ci = (-0.006694, -0.001954)   
S.JGRU   S.GRU         p = 1.000; ci = (-0.01685, -0.007300)   
         S.TRANS        p = 1.000; ci = (-0.02503, -0.01617)   
         S.JTRANS       p = 1.000; ci = (-0.02901, -0.02087)   
S.JTRANS S.GRU       ✷p = 3.151e-08; ci = (0.01016, 0.01578)   
         S.TRANS   ✷p = 5.483e-05; ci = (0.001871, 0.006547)   
         S.JGRU      ✷p = 3.151e-08; ci = (0.02084, 0.02902)   

                                                        M.TED  \
S.GRU    S.TRANS          p = 1.000; ci = (0.006643, 0.01820)   
         S.JGRU    ✷p = 1.378e-05; ci = (-0.02235, -0.007523)   
         S.JTRANS          p = 1.000; ci = (0.01395, 0.02448)   
S.TRANS  S.GRU     ✷p = 5.877e-06; ci = (-0.01827, -0.006593)   
         S.JGRU     ✷p = 3.663e-08; ci = (-0.03417, -0.02073)   
         S.JTRANS         p = 1.000; ci = (0.003000, 0.01023)   
S.JGRU   S.GRU            p = 1.000; ci = (0.007529, 0.02296)   
         S.TRANS           p = 1.000; ci = (0.02076, 0.03386)   
         S.JTRANS          p = 1.000; ci = (0.02795, 0.04000)   
S.JTRANS S.GRU      ✷p = 3.151e-08; ci = (-0.02445, -0.01396)   
         S.TRANS   ✷p = 3.703e-05; ci = (-0.01021, -0.002942)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.03983, -0.02767)   

                                                          M.TER  \
S.GRU    S.TRANS           p = 1.000; ci = (0.001959, 0.005368)   
         S.JGRU     ✷p = 1.378e-05; ci = (-0.006590, -0.002218)   
         S.JTRANS          p = 1.000; ci = (0.004115, 0.007221)   
S.TRANS  S.GRU      ✷p = 5.877e-06; ci = (-0.005390, -0.001945)   
         S.JGRU      ✷p = 3.663e-08; ci = (-0.01008, -0.006116)   
         S.JTRANS         p = 1.000; ci = (0.0008838, 0.003019)   
S.JGRU   S.GRU             p = 1.000; ci = (0.002221, 0.006773)   
         S.TRANS           p = 1.000; ci = (0.006123, 0.009987)   
         S.JTRANS           p = 1.000; ci = (0.008244, 0.01180)   
S.JTRANS S.GRU      ✷p = 3.151e-08; ci = (-0.007212, -0.004118)   
         S.TRANS   ✷p = 3.703e-05; ci = (-0.003012, -0.0008680)   
         S.JGRU      ✷p = 3.151e-08; ci = (-0.01175, -0.008165)   

                                                          M.FER  \
S.GRU    S.TRANS          p = 1.000; ci = (0.0004536, 0.001519)   
         S.JGRU    ✷p = 0.0002677; ci = (-0.001810, -0.0003404)   
         S.JTRANS          p = 1.000; ci = (0.001302, 0.002240)   
S.TRANS  S.GRU     ✷p = 3.122e-05; ci = (-0.001510, -0.0004630)   
         S.JGRU     ✷p = 1.192e-07; ci = (-0.002775, -0.001325)   
         S.JTRANS         p = 1.000; ci = (0.0003855, 0.001162)   
S.JGRU   S.GRU           p = 0.9997; ci = (0.0003455, 0.001872)   
         S.TRANS           p = 1.000; ci = (0.001332, 0.002757)   
         S.JTRANS          p = 1.000; ci = (0.002157, 0.003513)   
S.JTRANS S.GRU      ✷p = 3.151e-08; ci = (-0.002246, -0.001309)   
         S.TRANS   ✷p = 2.083e-05; ci = (-0.001152, -0.0003912)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.003518, -0.002155)   

                                                        M.BCF  
S.GRU    S.TRANS       p = 1.000; ci = (-0.008301, -0.003400)  
         S.JGRU     ✷p = 1.223e-05; ci = (0.003253, 0.009639)  
         S.JTRANS       p = 1.000; ci = (-0.01039, -0.005832)  
S.TRANS  S.GRU      ✷p = 1.868e-06; ci = (0.003369, 0.008296)  
         S.JGRU      ✷p = 3.151e-08; ci = (0.009432, 0.01507)  
         S.JTRANS    p = 0.9994; ci = (-0.003674, -0.0007476)  
S.JGRU   S.GRU         p = 1.000; ci = (-0.009771, -0.003204)  
         S.TRANS        p = 1.000; ci = (-0.01501, -0.009470)  
         S.JTRANS        p = 1.000; ci = (-0.01703, -0.0

<D.BAXTER: 'chinese_baxter'>

M.ACC  \
S.GRU    S.TRANS      p = 1.000; ci = (-0.04532, -0.02918)   
         S.JGRU    ✷p = 3.131e-06; ci = (0.01489, 0.03420)   
         S.JTRANS     p = 1.000; ci = (-0.04740, -0.03050)   
S.TRANS  S.GRU     ✷p = 3.663e-08; ci = (0.02937, 0.04548)   
         S.JGRU    ✷p = 3.151e-08; ci = (0.05072, 0.07324)   
         S.JTRANS    p = 0.6221; ci = (-0.01184, 0.008613)   
S.JGRU   S.GRU        p = 1.000; ci = (-0.03402, -0.01465)   
         S.TRANS      p = 1.000; ci = (-0.07329, -0.05047)   
         S.JTRANS     p = 1.000; ci = (-0.07499, -0.05183)   
S.JTRANS S.GRU     ✷p = 3.663e-08; ci = (0.03015, 0.04719)   
         S.TRANS     p = 0.3779; ci = (-0.008745, 0.01205)   
         S.JGRU    ✷p = 3.151e-08; ci = (0.05178, 0.07490)   

                                                        M.TED  \
S.GRU    S.TRANS         p = 0.9989; ci = (0.003727, 0.03601)   
         S.JGRU     ✷p = 1.151e-05; ci = (-0.07093, -0.02763)   
         S.JTRANS        p = 0.9998; ci = (0.009684, 0.04144)   
S.TRANS  S.GRU      ✷p = 0.001119; ci = (-0.03553, -0.004180)   
         S.JGRU     ✷p = 9.641e-07; ci = (-0.09490, -0.04415)   
         S.JTRANS        p = 0.8066; ci = (-0.01372, 0.02542)   
S.JGRU   S.GRU             p = 1.000; ci = (0.02719, 0.07109)   
         S.TRANS           p = 1.000; ci = (0.04443, 0.09422)   
         S.JTRANS          p = 1.000; ci = (0.04950, 0.09995)   
S.JTRANS S.GRU     ✷p = 0.0001974; ci = (-0.04162, -0.009776)   
         S.TRANS         p = 0.1934; ci = (-0.02560, 0.01470)   
         S.JGRU     ✷p = 3.708e-07; ci = (-0.09947, -0.04937)   

                                                        M.TER  \
S.GRU    S.TRANS       p = 0.9989; ci = (0.0009765, 0.009434)   
         S.JGRU    ✷p = 1.151e-05; ci = (-0.01858, -0.007240)   
         S.JTRANS        p = 0.9998; ci = (0.002539, 0.01086)   
S.TRANS  S.GRU     ✷p = 0.001119; ci = (-0.009307, -0.001095)   
         S.JGRU     ✷p = 9.641e-07; ci = (-0.02486, -0.01157)   
         S.JTRANS      p = 0.8066; ci = (-0.003594, 0.006659)   
S.JGRU   S.GRU            p = 1.000; ci = (0.007124, 0.01862)   
         S.TRANS           p = 1.000; ci = (0.01164, 0.02468)   
         S.JTRANS          p = 1.000; ci = (0.01297, 0.02619)   
S.JTRANS S.GRU     ✷p = 0.0001974; ci = (-0.01091, -0.002562)   
         S.TRANS       p = 0.1934; ci = (-0.006708, 0.003850)   
         S.JGRU     ✷p = 3.708e-07; ci = (-0.02606, -0.01293)   

                                                         M.FER  \
S.GRU    S.TRANS       p = 0.1337; ci = (-0.001727, 0.0006069)   
         S.JGRU    ✷p = 1.552e-05; ci = (-0.005733, -0.002224)   
         S.JTRANS      p = 0.9075; ci = (-0.0009053, 0.001441)   
S.TRANS  S.GRU         p = 0.8663; ci = (-0.0006405, 0.001725)   
         S.JGRU    ✷p = 0.0001605; ci = (-0.005438, -0.001487)   
         S.JTRANS      p = 0.9384; ci = (-0.0005589, 0.002299)   
S.JGRU   S.GRU            p = 1.000; ci = (0.002208, 0.005799)   
         S.TRANS         p = 0.9998; ci = (0.001405, 0.005413)   
         S.JTRANS         p = 1.000; ci = (0.002252, 0.006261)   
S.JTRANS S.GRU         p = 0.09251; ci = (-0.001435, 0.001000)   
         S.TRANS      p = 0.06155; ci = (-0.002326, 0.0006623)   
         S.JGRU    ✷p = 1.747e-05; ci = (-0.006209, -0.002354)   

                                                      M.BCF  
S.GRU    S.TRANS      p = 1.000; ci = (-0.01493, -0.004434)  
         S.JGRU    ✷p = 1.378e-05; ci = (0.009087, 0.02349)  
         S.JTRANS     p = 1.000; ci = (-0.01607, -0.005348)  
S.TRANS  S.GRU     ✷p = 1.965e-05; ci = (0.004555, 0.01497)  
         S.JGRU     ✷p = 2.114e-07; ci = (0.01802, 0.03428)  
         S.JTRANS    p = 0.6964; ci = (-0.007562, 0.005274)  
S.JGRU   S.GRU        p = 1.000; ci = (-0.02337, -0.009027)  
         S.TRANS       p = 1.000; ci = (-0.03409, -0.01809)  
         S.JTRANS      p = 1.000; ci = (-0.03532, -0.01874)  
S.JTRANS S.GRU     ✷p = 1.378e-05; ci = (0.005371, 0.01623)  
         S.TR

<D.NRO_IPA: 'Nromance_ipa'>

M.ACC  \
S.GRU    S.TRANS   p = 0.9848; ci = (-0.007551, 0.0009955)   
         S.JGRU     ✷p = 3.151e-08; ci = (0.09563, 0.2815)   
         S.JTRANS   p = 0.8066; ci = (-0.005057, 0.003383)   
S.TRANS  S.GRU     p = 0.01523; ci = (-0.001011, 0.007540)   
         S.JGRU     ✷p = 3.151e-08; ci = (0.09895, 0.2843)   
         S.JTRANS   p = 0.2085; ci = (-0.002965, 0.007566)   
S.JGRU   S.GRU         p = 1.000; ci = (-0.2892, -0.09588)   
         S.TRANS       p = 1.000; ci = (-0.2924, -0.09980)   
         S.JTRANS      p = 1.000; ci = (-0.2885, -0.09706)   
S.JTRANS S.GRU      p = 0.1934; ci = (-0.003506, 0.005273)   
         S.TRANS    p = 0.7915; ci = (-0.007742, 0.003080)   
         S.JGRU     ✷p = 3.151e-08; ci = (0.09634, 0.2810)   

                                                       M.TED  \
S.GRU    S.TRANS        p = 0.9983; ci = (0.002159, 0.01585)   
         S.JGRU       ✷p = 3.151e-08; ci = (-1.374, -0.2318)   
         S.JTRANS     p = 0.3132; ci = (-0.006410, 0.006389)   
S.TRANS  S.GRU     ✷p = 0.001668; ci = (-0.01596, -0.002200)   
         S.JGRU       ✷p = 3.151e-08; ci = (-1.381, -0.2415)   
         S.JTRANS  ✷p = 0.002667; ci = (-0.01671, -0.001658)   
S.JGRU   S.GRU               p = 1.000; ci = (0.2447, 1.398)   
         S.TRANS             p = 1.000; ci = (0.2544, 1.406)   
         S.JTRANS            p = 1.000; ci = (0.2443, 1.398)   
S.JTRANS S.GRU        p = 0.6868; ci = (-0.006421, 0.006537)   
         S.TRANS        p = 0.9973; ci = (0.001461, 0.01671)   
         S.JGRU       ✷p = 3.151e-08; ci = (-1.371, -0.2328)   

                                                         M.TER  \
S.GRU    S.TRANS        p = 0.9983; ci = (0.0002949, 0.002162)   
         S.JGRU       ✷p = 3.151e-08; ci = (-0.1874, -0.03161)   
         S.JTRANS     p = 0.3132; ci = (-0.0008747, 0.0008713)   
S.TRANS  S.GRU     ✷p = 0.001668; ci = (-0.002176, -0.0003000)   
         S.JGRU       ✷p = 3.151e-08; ci = (-0.1883, -0.03293)   
         S.JTRANS  ✷p = 0.002667; ci = (-0.002278, -0.0002261)   
S.JGRU   S.GRU               p = 1.000; ci = (0.03338, 0.1906)   
         S.TRANS             p = 1.000; ci = (0.03469, 0.1917)   
         S.JTRANS            p = 1.000; ci = (0.03331, 0.1906)   
S.JTRANS S.GRU        p = 0.6868; ci = (-0.0008754, 0.0008924)   
         S.TRANS        p = 0.9973; ci = (0.0001995, 0.002279)   
         S.JGRU       ✷p = 3.151e-08; ci = (-0.1870, -0.03174)   

                                                       M.FER  \
S.GRU    S.TRANS    p = 0.4143; ci = (-0.0004634, 0.0004130)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.1226, -0.02060)   
         S.JTRANS  p = 0.06155; ci = (-0.0005413, 0.0002346)   
S.TRANS  S.GRU      p = 0.5857; ci = (-0.0004006, 0.0004768)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.1225, -0.02055)   
         S.JTRANS   p = 0.1789; ci = (-0.0005636, 0.0002778)   
S.JGRU   S.GRU             p = 1.000; ci = (0.02135, 0.1162)   
         S.TRANS           p = 1.000; ci = (0.02127, 0.1160)   
         S.JTRANS          p = 1.000; ci = (0.02111, 0.1160)   
S.JTRANS S.GRU      p = 0.9384; ci = (-0.0002179, 0.0005431)   
         S.TRANS    p = 0.8211; ci = (-0.0002650, 0.0005713)   
         S.JGRU     ✷p = 3.151e-08; ci = (-0.1225, -0.02042)   

                                                       M.BCF  
S.GRU    S.TRANS    p = 0.9983; ci = (-0.003242, -0.0004878)  
         S.JGRU       ✷p = 3.151e-08; ci = (0.03924, 0.1708)  
         S.JTRANS     p = 0.3727; ci = (-0.001408, 0.001228)  
S.TRANS  S.GRU     ✷p = 0.001742; ci = (0.0005014, 0.003261)  
         S.JGRU       ✷p = 3.151e-08; ci = (0.04111, 0.1726)  
         S.JTRANS  ✷p = 0.004347; ci = (0.0002633, 0.003349)  
S.JGRU   S.GRU           p = 1.000; ci = (-0.1743, -0.04046)  
         S.TRANS         p = 1.000; ci = (-0.1760, -0.04226)  
         S.JTRANS        p = 1.000; ci = (-0.1744, -0.04042)  
S.JTRANS S.GRU        p = 0.6273; ci = (-0.001268, 0.001425)  
         S.TRANS    p = 0.99

<D.NRO_ORTO: 'Nromance_orto'>

M.ACC  \
S.GRU    S.TRANS      p = 0.9837; ci = (-0.005153, 0.001976)   
         S.JGRU      ✷p = 3.151e-08; ci = (0.05586, 0.08288)   
         S.JTRANS     p = 1.000; ci = (-0.009462, -0.002756)   
S.TRANS  S.GRU       p = 0.01630; ci = (-0.002106, 0.005201)   
         S.JGRU      ✷p = 3.151e-08; ci = (0.05770, 0.08503)   
         S.JTRANS   p = 0.9951; ci = (-0.008322, -0.0005181)   
S.JGRU   S.GRU          p = 1.000; ci = (-0.08264, -0.05552)   
         S.TRANS        p = 1.000; ci = (-0.08504, -0.05760)   
         S.JTRANS       p = 1.000; ci = (-0.08896, -0.06168)   
S.JTRANS S.GRU     ✷p = 4.906e-05; ci = (0.002705, 0.009404)   
         S.TRANS   ✷p = 0.004893; ci = (0.0003014, 0.008487)   
         S.JGRU      ✷p = 3.151e-08; ci = (0.06186, 0.08938)   

                                                     M.TED  \
S.GRU    S.TRANS    p = 0.9661; ci = (-0.001185, 0.009352)   
         S.JGRU    ✷p = 3.151e-08; ci = (-0.1649, -0.1257)   
         S.JTRANS   p = 0.9848; ci = (-0.0002850, 0.01105)   
S.TRANS  S.GRU     p = 0.03393; ci = (-0.009250, 0.001161)   
         S.JGRU    ✷p = 3.151e-08; ci = (-0.1690, -0.1299)   
         S.JTRANS   p = 0.6118; ci = (-0.003930, 0.006785)   
S.JGRU   S.GRU            p = 1.000; ci = (0.1246, 0.1651)   
         S.TRANS          p = 1.000; ci = (0.1294, 0.1689)   
         S.JTRANS         p = 1.000; ci = (0.1301, 0.1702)   
S.JTRANS S.GRU     p = 0.01523; ci = (-0.01090, 0.0002377)   
         S.TRANS    p = 0.3882; ci = (-0.006655, 0.004126)   
         S.JGRU    ✷p = 3.151e-08; ci = (-0.1702, -0.1308)   

                                                       M.TER M.FER  \
S.GRU    S.TRANS     p = 0.9661; ci = (-0.0001580, 0.001238)     -   
         S.JGRU    ✷p = 3.151e-08; ci = (-0.02183, -0.01664)     -   
         S.JTRANS    p = 0.9848; ci = (-3.709e-05, 0.001464)     -   
S.TRANS  S.GRU      p = 0.03393; ci = (-0.001225, 0.0001539)     -   
         S.JGRU    ✷p = 3.151e-08; ci = (-0.02238, -0.01720)     -   
         S.JTRANS   p = 0.6118; ci = (-0.0005207, 0.0008983)     -   
S.JGRU   S.GRU            p = 1.000; ci = (0.01650, 0.02186)     -   
         S.TRANS          p = 1.000; ci = (0.01713, 0.02237)     -   
         S.JTRANS         p = 1.000; ci = (0.01722, 0.02254)     -   
S.JTRANS S.GRU      p = 0.01523; ci = (-0.001443, 3.148e-05)     -   
         S.TRANS    p = 0.3882; ci = (-0.0008814, 0.0005460)     -   
         S.JGRU    ✷p = 3.151e-08; ci = (-0.02254, -0.01732)     -   

                                                       M.BCF  
S.GRU    S.TRANS    p = 0.9971; ci = (-0.002236, -0.0002569)  
         S.JGRU      ✷p = 3.151e-08; ci = (0.02177, 0.02808)  
         S.JTRANS   p = 0.9893; ci = (-0.002182, -3.093e-05)  
S.TRANS  S.GRU     ✷p = 0.002898; ci = (0.0002741, 0.002240)  
         S.JGRU      ✷p = 3.151e-08; ci = (0.02304, 0.02923)  
         S.JTRANS    p = 0.3132; ci = (-0.0008030, 0.001054)  
S.JGRU   S.GRU          p = 1.000; ci = (-0.02800, -0.02161)  
         S.TRANS        p = 1.000; ci = (-0.02920, -0.02299)  
         S.JTRANS       p = 1.000; ci = (-0.02913, -0.02279)  
S.JTRANS S.GRU       p = 0.01075; ci = (5.464e-05, 0.002151)  
         S.TRANS     p = 0.6868; ci = (-0.001082, 0.0007948)  
         S.JGRU      ✷p = 3.151e-08; ci = (0.02289, 0.02910)

### Summary Table

In [8]:
reflex_summary = summary_stats(T.REF, allD, [S.GRU, S.TRANS, S.JGRU, S.JTRANS])

In [9]:
# best in metric marking
for d in [D.WIKIHAN, D.WIKIHAN_A, D.BAXTER, D.NRO_IPA, D.NRO_ORTO]:
    for m in allM:
        perfs_for_m = []
        for s in [S.GRU, S.TRANS, S.JGRU, S.JTRANS]:
            perfs_for_m.append(((d, s), reflex_summary[(d, s)][m]))
        
        def f(A, B):
            ((d1, s1), r1) = A
            ((d2, s2), r2) = B
            val1 = r1['mean']
            val2 = r2['mean']
            if m in [M.ACC, M.BCF]:
                if val1 > val2:
                    return A
                else: 
                    return B
            else:
                if val1 > val2:
                    return B
                else: 
                    return A
                    
        best = reduce(f, perfs_for_m)
        ((_, best_s), _) = best
        reflex_summary[(d, best_s)][m]['best'] = True

In [10]:
reflex_summary_df = pd.DataFrame.from_dict(reflex_summary, orient='index')
reflex_summary_df_html = reflex_summary_df.copy()
for m in allM:
    reflex_summary_df_html[m] = reflex_summary_df_html[m].apply(format_summary(incl_std=True, incl_nsamples=True))
reflex_summary_df_html

M.ACC                   M.TED  \
D.WIKIHAN   S.GRU      20 | 0.6643 ± 0.0029    20 | 0.5244 ± 0.0049    
            S.TRANS    20 | 0.6639 ± 0.0053    20 | 0.5302 ± 0.0089    
            S.JGRU     20 | 0.6445 ± 0.0034    20 | 0.5558 ± 0.0060    
            S.JTRANS  20 | !0.6764 ± 0.0035   20 | !0.5128 ± 0.0072    
D.WIKIHAN_A S.GRU      20 | 0.6811 ± 0.0044    20 | 0.5007 ± 0.0083    
            S.TRANS    20 | 0.6896 ± 0.0036    20 | 0.4889 ± 0.0055    
            S.JGRU     20 | 0.6694 ± 0.0068    20 | 0.5159 ± 0.0101    
            S.JTRANS  20 | !0.6937 ± 0.0018   20 | !0.4826 ± 0.0028    
D.BAXTER    S.GRU      20 | 0.5172 ± 0.0070    20 | 0.7777 ± 0.0132    
            S.TRANS    20 | 0.5546 ± 0.0123    20 | 0.7576 ± 0.0243    
            S.JGRU     20 | 0.4926 ± 0.0157    20 | 0.8266 ± 0.0370    
            S.JTRANS  20 | !0.5560 ± 0.0130   20 | !0.7520 ± 0.0243    
D.NRO_IPA   S.GRU      20 | 0.6385 ± 0.0037    20 | 0.7439 ± 0.0068    
            S.TRANS   20 | !0.6419 ± 0.0064   20 | !0.7349 ± 0.0096    
            S.JGRU     20 | 0.4828 ± 0.1482    20 | 1.3257 ± 0.8784    
            S.JTRANS   20 | 0.6396 ± 0.0065    20 | 0.7442 ± 0.0087    
D.NRO_ORTO  S.GRU      20 | 0.6458 ± 0.0034    20 | 0.7301 ± 0.0069    
            S.TRANS    20 | 0.6480 ± 0.0050    20 | 0.7258 ± 0.0061    
            S.JGRU     20 | 0.5792 ± 0.0231    20 | 0.8741 ± 0.0346    
            S.JTRANS  20 | !0.6520 ± 0.0046   20 | !0.7247 ± 0.0069    

                                       M.TER                   M.FER  \
D.WIKIHAN   S.GRU      20 | 0.1547 ± 0.0014    20 | 0.0400 ± 0.0006    
            S.TRANS    20 | 0.1564 ± 0.0026    20 | 0.0406 ± 0.0007    
            S.JGRU     20 | 0.1640 ± 0.0018    20 | 0.0428 ± 0.0007    
            S.JTRANS  20 | !0.1513 ± 0.0021   20 | !0.0390 ± 0.0006    
D.WIKIHAN_A S.GRU      20 | 0.1477 ± 0.0024    20 | 0.0380 ± 0.0007    
            S.TRANS    20 | 0.1442 ± 0.0016    20 | 0.0371 ± 0.0006    
            S.JGRU     20 | 0.1522 ± 0.0030    20 | 0.0391 ± 0.0011    
            S.JTRANS  20 | !0.1424 ± 0.0008   20 | !0.0363 ± 0.0004    
D.BAXTER    S.GRU      20 | 0.2037 ± 0.0035    20 | 0.0488 ± 0.0010    
            S.TRANS    20 | 0.1985 ± 0.0064    20 | 0.0494 ± 0.0018    
            S.JGRU     20 | 0.2166 ± 0.0097    20 | 0.0528 ± 0.0030    
            S.JTRANS  20 | !0.1970 ± 0.0064   20 | !0.0485 ± 0.0018    
D.NRO_IPA   S.GRU      20 | 0.1014 ± 0.0009   20 | !0.0426 ± 0.0005    
            S.TRANS   20 | !0.1002 ± 0.0013    20 | 0.0427 ± 0.0006    
            S.JGRU     20 | 0.1808 ± 0.1198    20 | 0.0930 ± 0.0748    
            S.JTRANS   20 | 0.1015 ± 0.0012    20 | 0.0428 ± 0.0005    
D.NRO_ORTO  S.GRU      20 | 0.0967 ± 0.0009                        -   
            S.TRANS    20 | 0.0961 ± 0.0008                        -   
            S.JGRU     20 | 0.1158 ± 0.0046                        -   
            S.JTRANS  20 | !0.0960 ± 0.0009                        -   

                                       M.BCF  
D.WIKIHAN   S.GRU      20 | 0.7394 ± 0.0022   
            S.TRANS    20 | 0.7370 ± 0.0040   
            S.JGRU     20 | 0.7260 ± 0.0027   
            S.JTRANS  20 | !0.7445 ± 0.0031   
D.WIKIHAN_A S.GRU      20 | 0.7495 ± 0.0036   
            S.TRANS    20 | 0.7551 ± 0.0022   
            S.JGRU     20 | 0.7430 ± 0.0043   
            S.JTRANS  20 | !0.7572 ± 0.0013   
D.BAXTER    S.GRU      20 | 0.6783 ± 0.0046   
            S.TRANS    20 | 0.6882 ± 0.0078   
            S.JGRU     20 | 0.6622 ± 0.0120   
            S.JTRANS  20 | !0.6892 ± 0.0081   
D.NRO_IPA   S.GRU      20 | 0.8361 ± 0.0014   
            S.TRANS   20 | !0.8380 ± 0.0019   
            S.JGRU     20 | 0.7567 ± 0.1035   
            S.JTRANS   20 | 0.8361 ± 0.0018   
D.NRO_ORTO  S.GRU      20 | 0.8465 ± 0.0014   
            S.TRANS   20 | !0.8478 ± 0.0011   
            S.JGRU     20 | 0.8218 ± 0.0054   
            S.JTRANS   20 | 0.8476 ± 0.0012

## Reconstruction

In [11]:
recon_include_ss = [S.GRU, S.TRANS, S.GRUBS, S.GRUBS_GRU, S.GRUBS_JTRANS]

In [12]:
recon_pairs = get_pairs(recon_include_ss)
recon_pairs

[(<S.GRU: 'GRU'>, <S.TRANS: 'Transformer'>),
 (<S.GRU: 'GRU'>, <S.GRUBS: 'GRU-BS'>),
 (<S.GRU: 'GRU'>, <S.GRUBS_GRU: 'GRU-BS+GRU'>),
 (<S.GRU: 'GRU'>, <S.GRUBS_JTRANS: 'GRU-BS+JambuTransformer'>),
 (<S.TRANS: 'Transformer'>, <S.GRU: 'GRU'>),
 (<S.TRANS: 'Transformer'>, <S.GRUBS: 'GRU-BS'>),
 (<S.TRANS: 'Transformer'>, <S.GRUBS_GRU: 'GRU-BS+GRU'>),
 (<S.TRANS: 'Transformer'>, <S.GRUBS_JTRANS: 'GRU-BS+JambuTransformer'>),
 (<S.GRUBS: 'GRU-BS'>, <S.GRU: 'GRU'>),
 (<S.GRUBS: 'GRU-BS'>, <S.TRANS: 'Transformer'>),
 (<S.GRUBS: 'GRU-BS'>, <S.GRUBS_GRU: 'GRU-BS+GRU'>),
 (<S.GRUBS: 'GRU-BS'>, <S.GRUBS_JTRANS: 'GRU-BS+JambuTransformer'>),
 (<S.GRUBS_GRU: 'GRU-BS+GRU'>, <S.GRU: 'GRU'>),
 (<S.GRUBS_GRU: 'GRU-BS+GRU'>, <S.TRANS: 'Transformer'>),
 (<S.GRUBS_GRU: 'GRU-BS+GRU'>, <S.GRUBS: 'GRU-BS'>),
 (<S.GRUBS_GRU: 'GRU-BS+GRU'>, <S.GRUBS_JTRANS: 'GRU-BS+JambuTransformer'>),
 (<S.GRUBS_JTRANS: 'GRU-BS+JambuTransformer'>, <S.GRU: 'GRU'>),
 (<S.GRUBS_JTRANS: 'GRU-BS+JambuTransformer'>, <S.TRANS: 'Transf

In [ ]:
recon_stats_res: dict[D, dict[tuple[S, S], dict[M, dict]]] = {}
for d in allD:
    recon_stats_res[d] = run_tests(T.REC, d, recon_pairs)

### Statistical Test Tables

In [14]:
recon_stats_dfs = {}
for d in allD:
    df = pd.DataFrame.from_dict(recon_stats_res[d], orient='index')
    # df = df.reindex(columns=[M.ACC, M.TED, M.TER, M.FER, M.BCF])
    recon_stats_dfs[d] = df

In [15]:
for d in allD:
    display(d)
    tmp_df = recon_stats_dfs[d].copy()
    for m in allM:
        tmp_df[m] = tmp_df[m].apply(format_stats)
    display(tmp_df)

<D.WIKIHAN: 'chinese_wikihan2022'>

M.ACC  \
S.GRU          S.TRANS          ✷p = 0.007456; ci = (0.001113, 0.01849)   
               S.GRUBS          p = 0.04052; ci = (-0.0009273, 0.01515)   
               S.GRUBS_GRU        p = 1.000; ci = (-0.02227, -0.008975)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.02396, -0.01014)   
S.TRANS        S.GRU             p = 0.9925; ci = (-0.01825, -0.001355)   
               S.GRUBS            p = 0.8066; ci = (-0.01236, 0.006341)   
               S.GRUBS_GRU         p = 1.000; ci = (-0.03393, -0.01728)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.03545, -0.01834)   
S.GRUBS        S.GRU             p = 0.9595; ci = (-0.01500, 0.0006776)   
               S.TRANS            p = 0.1934; ci = (-0.006776, 0.01216)   
               S.GRUBS_GRU         p = 1.000; ci = (-0.03042, -0.01500)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.03167, -0.01620)   
S.GRUBS_GRU    S.GRU           ✷p = 5.877e-06; ci = (0.008955, 0.02241)   
               S.TRANS          ✷p = 2.613e-07; ci = (0.01718, 0.03393)   
               S.GRUBS          ✷p = 3.974e-07; ci = (0.01508, 0.03030)   
               S.GRUBS_JTRANS    p = 0.6723; ci = (-0.007986, 0.005276)   
S.GRUBS_JTRANS S.GRU           ✷p = 3.131e-06; ci = (0.009923, 0.02370)   
               S.TRANS          ✷p = 2.269e-07; ci = (0.01830, 0.03558)   
               S.GRUBS          ✷p = 2.436e-07; ci = (0.01597, 0.03141)   
               S.GRUBS_GRU       p = 0.3277; ci = (-0.005373, 0.007986)   

                                                                   M.TED  \
S.GRU          S.TRANS            p = 0.02826; ci = (-0.02167, 0.002944)   
               S.GRUBS         ✷p = 0.004014; ci = (-0.02885, -0.001839)   
               S.GRUBS_GRU            p = 1.000; ci = (0.01960, 0.04279)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.02159, 0.04641)   
S.TRANS        S.GRU               p = 0.9717; ci = (-0.002323, 0.02222)   
               S.GRUBS             p = 0.1427; ci = (-0.01970, 0.009245)   
               S.GRUBS_GRU            p = 1.000; ci = (0.02798, 0.05332)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.03015, 0.05703)   
S.GRUBS        S.GRU                p = 0.9960; ci = (0.001646, 0.02861)   
               S.TRANS             p = 0.8573; ci = (-0.008470, 0.01970)   
               S.GRUBS_GRU            p = 1.000; ci = (0.03272, 0.06075)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.03437, 0.06261)   
S.GRUBS_GRU    S.GRU           ✷p = 1.750e-06; ci = (-0.04261, -0.01926)   
               S.TRANS         ✷p = 1.969e-07; ci = (-0.05368, -0.02807)   
               S.GRUBS         ✷p = 1.590e-07; ci = (-0.05983, -0.03214)   
               S.GRUBS_JTRANS       p = 0.5962; ci = (-0.01021, 0.01583)   
S.GRUBS_JTRANS S.GRU           ✷p = 7.878e-07; ci = (-0.04560, -0.02130)   
               S.TRANS         ✷p = 1.377e-07; ci = (-0.05644, -0.02967)   
               S.GRUBS         ✷p = 1.031e-07; ci = (-0.06252, -0.03383)   
               S.GRUBS_GRU          p = 0.4038; ci = (-0.01502, 0.01002)   

                                                                     M.TER  \
S.GRU          S.TRANS            p = 0.02826; ci = (-0.005070, 0.0006959)   
               S.GRUBS         ✷p = 0.004014; ci = (-0.006758, -0.0004311)   
               S.GRUBS_GRU             p = 1.000; ci = (0.004592, 0.01002)   
               S.GRUBS_JTRANS          p = 1.000; ci = (0.005057, 0.01086)   
S.TRANS        S.GRU               p = 0.9717; ci = (-0.0005442, 0.005204)   
               S.GRUBS              p = 0.1427; ci = (-0.004615, 0.002166)   
               S.GRUBS_GRU             p = 1.000; ci = (0.006553, 0.01248)   
               S.GRUBS_JTRANS          p = 1.000; ci = (0.007060, 0.01335)   
S.GRUBS        S.GRU                p = 0.9960; ci = (0.0003805, 0.006697)   
               S.TRANS              p = 0.8573; ci = (-0.001984, 0.004615)   
               S.GRUBS_GRU         

<D.WIKIHAN_A: 'chinese_wikihan2022_augmented'>

M.ACC  \
S.GRU          S.TRANS           p = 0.9998; ci = (-0.01709, -0.002517)   
               S.GRUBS             p = 1.000; ci = (-0.02454, -0.01215)   
               S.GRUBS_GRU         p = 1.000; ci = (-0.04419, -0.03146)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.04436, -0.03127)   
S.TRANS        S.GRU           ✷p = 0.0002186; ci = (0.002227, 0.01713)   
               S.GRUBS           p = 0.9971; ci = (-0.01600, -0.002323)   
               S.GRUBS_GRU         p = 1.000; ci = (-0.03576, -0.02159)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.03601, -0.02125)   
S.GRUBS        S.GRU            ✷p = 6.004e-07; ci = (0.01178, 0.02469)   
               S.TRANS          ✷p = 0.002898; ci = (0.002227, 0.01582)   
               S.GRUBS_GRU         p = 1.000; ci = (-0.02508, -0.01409)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.02531, -0.01389)   
S.GRUBS_GRU    S.GRU            ✷p = 3.398e-08; ci = (0.03122, 0.04424)   
               S.TRANS          ✷p = 3.398e-08; ci = (0.02164, 0.03540)   
               S.GRUBS          ✷p = 5.733e-08; ci = (0.01413, 0.02488)   
               S.GRUBS_JTRANS    p = 0.3882; ci = (-0.005760, 0.005787)   
S.GRUBS_JTRANS S.GRU            ✷p = 3.663e-08; ci = (0.03115, 0.04439)   
               S.TRANS          ✷p = 3.663e-08; ci = (0.02139, 0.03546)   
               S.GRUBS          ✷p = 5.733e-08; ci = (0.01369, 0.02523)   
               S.GRUBS_GRU       p = 0.6118; ci = (-0.005808, 0.005760)   

                                                                   M.TED  \
S.GRU          S.TRANS                p = 1.000; ci = (0.01200, 0.03621)   
               S.GRUBS                p = 1.000; ci = (0.02529, 0.04487)   
               S.GRUBS_GRU            p = 1.000; ci = (0.06334, 0.08538)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.06233, 0.08285)   
S.TRANS        S.GRU           ✷p = 3.305e-05; ci = (-0.03606, -0.01252)   
               S.GRUBS             p = 0.9595; ci = (0.0001264, 0.02260)   
               S.GRUBS_GRU            p = 1.000; ci = (0.03752, 0.06288)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.03669, 0.06072)   
S.GRUBS        S.GRU           ✷p = 1.590e-07; ci = (-0.04501, -0.02512)   
               S.TRANS          p = 0.04052; ci = (-0.02303, -9.680e-05)   
               S.GRUBS_GRU            p = 1.000; ci = (0.02846, 0.04960)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.02756, 0.04748)   
S.GRUBS_GRU    S.GRU           ✷p = 3.151e-08; ci = (-0.08533, -0.06312)   
               S.TRANS         ✷p = 4.941e-08; ci = (-0.06291, -0.03804)   
               S.GRUBS         ✷p = 1.031e-07; ci = (-0.04886, -0.02823)   
               S.GRUBS_JTRANS      p = 0.3625; ci = (-0.01225, 0.009696)   
S.GRUBS_JTRANS S.GRU           ✷p = 3.151e-08; ci = (-0.08298, -0.06201)   
               S.TRANS         ✷p = 3.663e-08; ci = (-0.06045, -0.03665)   
               S.GRUBS         ✷p = 7.154e-08; ci = (-0.04676, -0.02752)   
               S.GRUBS_GRU         p = 0.6375; ci = (-0.009183, 0.01302)   

                                                                     M.TER  \
S.GRU          S.TRANS                p = 1.000; ci = (0.002812, 0.008486)   
               S.GRUBS                 p = 1.000; ci = (0.005921, 0.01051)   
               S.GRUBS_GRU              p = 1.000; ci = (0.01484, 0.02000)   
               S.GRUBS_JTRANS           p = 1.000; ci = (0.01462, 0.01941)   
S.TRANS        S.GRU           ✷p = 3.305e-05; ci = (-0.008447, -0.002933)   
               S.GRUBS              p = 0.9595; ci = (2.169e-05, 0.005294)   
               S.GRUBS_GRU             p = 1.000; ci = (0.008787, 0.01473)   
               S.GRUBS_JTRANS          p = 1.000; ci = (0.008594, 0.01423)   
S.GRUBS        S.GRU            ✷p = 1.590e-07; ci = (-0.01054, -0.005884)   
               S.TRANS           p = 0.04052; ci = (-0.005393, -2.267e-05)   
               S.GRUBS_GRU         

<D.BAXTER: 'chinese_baxter'>

M.ACC  \
S.GRU          S.TRANS             p = 1.000; ci = (-0.06584, -0.02236)   
               S.GRUBS           p = 0.9991; ci = (-0.05155, -0.005901)   
               S.GRUBS_GRU         p = 1.000; ci = (-0.08128, -0.03696)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.09876, -0.05404)   
S.TRANS        S.GRU            ✷p = 3.305e-05; ci = (0.02298, 0.06584)   
               S.GRUBS           p = 0.04676; ci = (-0.008696, 0.04046)   
               S.GRUBS_GRU        p = 0.9119; ci = (-0.03975, 0.009627)   
               S.GRUBS_JTRANS    p = 0.9969; ci = (-0.05621, -0.008585)   
S.GRUBS        S.GRU           ✷p = 0.0009330; ci = (0.005590, 0.05155)   
               S.TRANS            p = 0.9532; ci = (-0.04099, 0.008385)   
               S.GRUBS_GRU       p = 0.9963; ci = (-0.05714, -0.005901)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.07360, -0.02360)   
S.GRUBS_GRU    S.GRU            ✷p = 2.422e-06; ci = (0.03696, 0.08168)   
               S.TRANS           p = 0.08811; ci = (-0.009317, 0.03851)   
               S.GRUBS          ✷p = 0.003704; ci = (0.004178, 0.05559)   
               S.GRUBS_JTRANS     p = 0.9029; ci = (-0.04317, 0.006656)   
S.GRUBS_JTRANS S.GRU            ✷p = 4.256e-08; ci = (0.05482, 0.09783)   
               S.TRANS          ✷p = 0.003147; ci = (0.008075, 0.05590)   
               S.GRUBS          ✷p = 3.703e-05; ci = (0.02244, 0.07236)   
               S.GRUBS_GRU       p = 0.09707; ci = (-0.008385, 0.04037)   

                                                                   M.TED  \
S.GRU          S.TRANS                 p = 1.000; ci = (0.05714, 0.1453)   
               S.GRUBS             p = 0.9914; ci = (-0.0009317, 0.1053)   
               S.GRUBS_GRU             p = 1.000; ci = (0.07391, 0.1652)   
               S.GRUBS_JTRANS          p = 1.000; ci = (0.09348, 0.1891)   
S.TRANS        S.GRU            ✷p = 8.503e-06; ci = (-0.1463, -0.05714)   
               S.GRUBS           p = 0.01422; ci = (-0.09348, -0.002795)   
               S.GRUBS_GRU          p = 0.8416; ci = (-0.01863, 0.05621)   
               S.GRUBS_JTRANS       p = 0.9920; ci = (0.003106, 0.08043)   
S.GRUBS        S.GRU            ✷p = 0.008647; ci = (-0.1048, -0.002675)   
               S.TRANS              p = 0.9858; ci = (0.002484, 0.09441)   
               S.GRUBS_GRU            p = 0.9985; ci = (0.02112, 0.1130)   
               S.GRUBS_JTRANS         p = 0.9999; ci = (0.04161, 0.1381)   
S.GRUBS_GRU    S.GRU            ✷p = 3.556e-06; ci = (-0.1638, -0.07432)   
               S.TRANS              p = 0.1584; ci = (-0.05528, 0.01863)   
               S.GRUBS           ✷p = 0.001463; ci = (-0.1130, -0.01887)   
               S.GRUBS_JTRANS       p = 0.8663; ci = (-0.01692, 0.06394)   
S.GRUBS_JTRANS S.GRU            ✷p = 2.804e-07; ci = (-0.1883, -0.09504)   
               S.TRANS         ✷p = 0.008032; ci = (-0.08012, -0.002094)   
               S.GRUBS          ✷p = 0.0001235; ci = (-0.1373, -0.04037)   
               S.GRUBS_GRU          p = 0.1337; ci = (-0.06304, 0.01849)   

                                                                    M.TER  \
S.GRU          S.TRANS                 p = 1.000; ci = (0.01389, 0.03483)   
               S.GRUBS               p = 0.9975; ci = (0.002626, 0.02686)   
               S.GRUBS_GRU             p = 1.000; ci = (0.01932, 0.04035)   
               S.GRUBS_JTRANS          p = 1.000; ci = (0.02368, 0.04553)   
S.TRANS        S.GRU            ✷p = 6.651e-06; ci = (-0.03479, -0.01397)   
               S.GRUBS             p = 0.01990; ci = (-0.01986, 0.001080)   
               S.GRUBS_GRU          p = 0.9029; ci = (-0.003243, 0.01426)   
               S.GRUBS_JTRANS        p = 0.9907; ci = (0.001745, 0.01970)   
S.GRUBS        S.GRU            ✷p = 0.002452; ci = (-0.02682, -0.003449)   
               S.TRANS              p = 0.9801; ci = (-0.001083, 0.02020)   
               S.GRUBS_GRU           p = 0.998

<D.NRO_IPA: 'Nromance_ipa'>

M.ACC  \
S.GRU          S.TRANS           p = 0.9999; ci = (-0.01742, -0.005171)   
               S.GRUBS           p = 0.9979; ci = (-0.01244, -0.001293)   
               S.GRUBS_GRU         p = 1.000; ci = (-0.02599, -0.01425)   
               S.GRUBS_JTRANS      p = 1.000; ci = (-0.02520, -0.01308)   
S.TRANS        S.GRU           ✷p = 5.483e-05; ci = (0.005331, 0.01736)   
               S.GRUBS           p = 0.08811; ci = (-0.001722, 0.01031)   
               S.GRUBS_GRU       p = 0.9992; ci = (-0.01531, -0.002509)   
               S.GRUBS_JTRANS    p = 0.9973; ci = (-0.01440, -0.001511)   
S.GRUBS        S.GRU            ✷p = 0.002070; ci = (0.001682, 0.01231)   
               S.TRANS            p = 0.9119; ci = (-0.01023, 0.001693)   
               S.GRUBS_GRU        p = 1.000; ci = (-0.01890, -0.007469)   
               S.GRUBS_JTRANS     p = 1.000; ci = (-0.01816, -0.006328)   
S.GRUBS_GRU    S.GRU            ✷p = 1.590e-07; ci = (0.01462, 0.02598)   
               S.TRANS         ✷p = 0.0007758; ci = (0.002668, 0.01543)   
               S.GRUBS         ✷p = 9.604e-06; ci = (0.007554, 0.01913)   
               S.GRUBS_JTRANS    p = 0.3474; ci = (-0.005287, 0.007185)   
S.GRUBS_JTRANS S.GRU            ✷p = 1.377e-07; ci = (0.01358, 0.02517)   
               S.TRANS          ✷p = 0.002667; ci = (0.001440, 0.01459)   
               S.GRUBS         ✷p = 2.480e-05; ci = (0.006490, 0.01816)   
               S.GRUBS_GRU       p = 0.6526; ci = (-0.007070, 0.005245)   

                                                                   M.TED  \
S.GRU          S.TRANS                p = 1.000; ci = (0.05673, 0.08863)   
               S.GRUBS                p = 1.000; ci = (0.04875, 0.08097)   
               S.GRUBS_GRU             p = 1.000; ci = (0.08461, 0.1159)   
               S.GRUBS_JTRANS          p = 1.000; ci = (0.08435, 0.1167)   
S.TRANS        S.GRU           ✷p = 3.151e-08; ci = (-0.08902, -0.05701)   
               S.GRUBS             p = 0.1280; ci = (-0.02218, 0.005481)   
               S.GRUBS_GRU            p = 1.000; ci = (0.01405, 0.04078)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.01359, 0.04136)   
S.GRUBS        S.GRU           ✷p = 5.733e-08; ci = (-0.08108, -0.04895)   
               S.TRANS             p = 0.8720; ci = (-0.005736, 0.02161)   
               S.GRUBS_GRU            p = 1.000; ci = (0.02180, 0.04852)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.02115, 0.04949)   
S.GRUBS_GRU    S.GRU            ✷p = 3.151e-08; ci = (-0.1160, -0.08454)   
               S.TRANS         ✷p = 2.480e-05; ci = (-0.04102, -0.01408)   
               S.GRUBS         ✷p = 1.259e-06; ci = (-0.04900, -0.02189)   
               S.GRUBS_JTRANS       p = 0.6066; ci = (-0.01333, 0.01373)   
S.GRUBS_JTRANS S.GRU            ✷p = 3.151e-08; ci = (-0.1165, -0.08430)   
               S.TRANS         ✷p = 3.499e-05; ci = (-0.04169, -0.01380)   
               S.GRUBS         ✷p = 3.337e-06; ci = (-0.05011, -0.02178)   
               S.GRUBS_GRU          p = 0.3934; ci = (-0.01390, 0.01321)   

                                                                     M.TER  \
S.GRU          S.TRANS                 p = 1.000; ci = (0.007696, 0.01155)   
               S.GRUBS                  p = 1.000; ci = (0.02063, 0.02463)   
               S.GRUBS_GRU              p = 1.000; ci = (0.02458, 0.02845)   
               S.GRUBS_JTRANS           p = 1.000; ci = (0.02460, 0.02857)   
S.TRANS        S.GRU            ✷p = 3.151e-08; ci = (-0.01162, -0.007736)   
               S.GRUBS                  p = 1.000; ci = (0.01142, 0.01447)   
               S.GRUBS_GRU              p = 1.000; ci = (0.01546, 0.01839)   
               S.GRUBS_JTRANS           p = 1.000; ci = (0.01540, 0.01849)   
S.GRUBS        S.GRU             ✷p = 3.151e-08; ci = (-0.02464, -0.02060)   
               S.TRANS           ✷p = 3.151e-08; ci = (-0.01452, -0.01146)   
               S.GRUBS_GRU         

<D.NRO_ORTO: 'Nromance_orto'>

M.ACC  \
S.GRU          S.TRANS            p = 1.000; ci = (-0.02144, -0.01278)   
               S.GRUBS            p = 1.000; ci = (-0.02166, -0.01301)   
               S.GRUBS_GRU        p = 1.000; ci = (-0.03656, -0.02870)   
               S.GRUBS_JTRANS     p = 1.000; ci = (-0.03533, -0.02736)   
S.TRANS        S.GRU           ✷p = 3.151e-08; ci = (0.01275, 0.02160)   
               S.GRUBS          p = 0.7419; ci = (-0.004031, 0.004112)   
               S.GRUBS_GRU        p = 1.000; ci = (-0.01893, -0.01149)   
               S.GRUBS_JTRANS     p = 1.000; ci = (-0.01785, -0.01009)   
S.GRUBS        S.GRU           ✷p = 3.663e-08; ci = (0.01295, 0.02144)   
               S.TRANS          p = 0.2581; ci = (-0.004059, 0.004277)   
               S.GRUBS_GRU        p = 1.000; ci = (-0.01912, -0.01152)   
               S.GRUBS_JTRANS     p = 1.000; ci = (-0.01798, -0.01021)   
S.GRUBS_GRU    S.GRU           ✷p = 3.151e-08; ci = (0.02864, 0.03653)   
               S.TRANS         ✷p = 7.700e-08; ci = (0.01164, 0.01897)   
               S.GRUBS         ✷p = 4.941e-08; ci = (0.01148, 0.01898)   
               S.GRUBS_JTRANS   p = 0.3779; ci = (-0.002247, 0.004688)   
S.GRUBS_JTRANS S.GRU           ✷p = 3.151e-08; ci = (0.02736, 0.03528)   
               S.TRANS         ✷p = 2.114e-07; ci = (0.01026, 0.01795)   
               S.GRUBS         ✷p = 1.031e-07; ci = (0.01018, 0.01795)   
               S.GRUBS_GRU      p = 0.6221; ci = (-0.004689, 0.002230)   

                                                                   M.TED  \
S.GRU          S.TRANS                p = 1.000; ci = (0.02493, 0.04953)   
               S.GRUBS                p = 1.000; ci = (0.03788, 0.05946)   
               S.GRUBS_GRU            p = 1.000; ci = (0.06824, 0.08816)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.06724, 0.08694)   
S.TRANS        S.GRU           ✷p = 3.708e-07; ci = (-0.04928, -0.02536)   
               S.GRUBS             p = 0.9848; ci = (-0.001127, 0.02218)   
               S.GRUBS_GRU            p = 1.000; ci = (0.02927, 0.05148)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.02810, 0.05062)   
S.GRUBS        S.GRU           ✷p = 3.663e-08; ci = (-0.05897, -0.03795)   
               S.TRANS           p = 0.01523; ci = (-0.02256, 0.0008367)   
               S.GRUBS_GRU            p = 1.000; ci = (0.02013, 0.03888)   
               S.GRUBS_JTRANS         p = 1.000; ci = (0.01907, 0.03804)   
S.GRUBS_GRU    S.GRU           ✷p = 3.151e-08; ci = (-0.08816, -0.06842)   
               S.TRANS         ✷p = 7.700e-08; ci = (-0.05175, -0.02950)   
               S.GRUBS         ✷p = 3.007e-07; ci = (-0.03896, -0.01993)   
               S.GRUBS_JTRANS     p = 0.4409; ci = (-0.009663, 0.007924)   
S.GRUBS_JTRANS S.GRU           ✷p = 3.151e-08; ci = (-0.08750, -0.06744)   
               S.TRANS         ✷p = 8.914e-08; ci = (-0.05091, -0.02819)   
               S.GRUBS         ✷p = 5.236e-07; ci = (-0.03804, -0.01895)   
               S.GRUBS_GRU        p = 0.5591; ci = (-0.007719, 0.009966)   

                                                                     M.TER  \
S.GRU          S.TRANS                p = 1.000; ci = (0.003037, 0.006366)   
               S.GRUBS                  p = 1.000; ci = (0.01526, 0.01798)   
               S.GRUBS_GRU              p = 1.000; ci = (0.01862, 0.02121)   
               S.GRUBS_JTRANS           p = 1.000; ci = (0.01851, 0.02104)   
S.TRANS        S.GRU           ✷p = 1.437e-06; ci = (-0.006386, -0.003057)   
               S.GRUBS                  p = 1.000; ci = (0.01031, 0.01325)   
               S.GRUBS_GRU              p = 1.000; ci = (0.01370, 0.01653)   
               S.GRUBS_JTRANS           p = 1.000; ci = (0.01356, 0.01644)   
S.GRUBS        S.GRU             ✷p = 3.151e-08; ci = (-0.01792, -0.01521)   
               S.TRANS           ✷p = 3.151e-08; ci = (-0.01334, -0.01033)   
               S.GRUBS_GRU            p = 1.000; ci = (

### Summary Table

In [16]:
recon_summary = summary_stats(T.REC, allD, recon_include_ss)

In [17]:
# significance markings
baselines = [S.GRU, S.TRANS]
for d in [D.WIKIHAN, D.WIKIHAN_A, D.BAXTER, D.NRO_IPA, D.NRO_ORTO]:
    for s in recon_include_ss:
        for m in allM:
            
            # baselines
            if s not in baselines:
                sig = 0
                for b in baselines:
                    if recon_stats_res[d][(s, b)][m]['sig_better'] == True:
                        sig += 1
                if sig == len(baselines):
                    recon_summary[(d, s)][m]['sig_better_baseline'] = True
            
            # ablation
            if s in [S.GRUBS_GRU, S.GRUBS_TRANS, S.GRUBS_JTRANS]:
                b = S.GRUBS
                if recon_stats_res[d][(s, b)][m]['sig_better'] == True:
                    recon_summary[(d, s)][m]['sig_better_ablation'] = True

In [18]:
# best in metric marking
for d in [D.WIKIHAN, D.WIKIHAN_A, D.BAXTER, D.NRO_IPA, D.NRO_ORTO]:
    for m in allM:
        perfs_for_m = []
        for s in recon_include_ss:
            perfs_for_m.append(((d, s), recon_summary[(d, s)][m]))
        
        def f(A, B):
            ((d1, s1), r1) = A
            ((d2, s2), r2) = B
            val1 = r1['mean']
            val2 = r2['mean']
            if m in [M.ACC, M.BCF]:
                if val1 > val2:
                    return A
                else: 
                    return B
            else:
                if val1 > val2:
                    return B
                else: 
                    return A
                    
        best = reduce(f, perfs_for_m)
        ((_, best_s), _) = best
        recon_summary[(d, best_s)][m]['best'] = True

In [19]:
recon_summary_df = pd.DataFrame.from_dict(recon_summary, orient='index')
recon_summary_df_html = recon_summary_df.copy()
for m in allM:
    recon_summary_df_html[m] = recon_summary_df_html[m].apply(format_summary(incl_std=True, incl_nsamples=False))
recon_summary_df_html

M.ACC                M.TED  \
D.WIKIHAN   S.GRU              0.5558 ± 0.0086      0.7360 ± 0.0137    
            S.TRANS            0.5462 ± 0.0122      0.7453 ± 0.0165    
            S.GRUBS            0.5488 ± 0.0107      0.7507 ± 0.0186    
            S.GRUBS_GRU      0.5714 ± 0.0080 *†   0.7045 ± 0.0146 *†   
            S.GRUBS_JTRANS  !0.5726 ± 0.0083 *†  !0.7029 ± 0.0161 *†   
D.WIKIHAN_A S.GRU              0.5473 ± 0.0084      0.7574 ± 0.0127    
            S.TRANS            0.5582 ± 0.0097      0.7317 ± 0.0165    
            S.GRUBS           0.5664 ± 0.0066 *     0.7214 ± 0.0113    
            S.GRUBS_GRU     !0.5858 ± 0.0070 *†  !0.6822 ± 0.0143 *†   
            S.GRUBS_JTRANS   0.5858 ± 0.0075 *†   0.6840 ± 0.0129 *†   
D.BAXTER    S.GRU              0.3463 ± 0.0237      1.0916 ± 0.0629    
            S.TRANS            0.3901 ± 0.0289      0.9904 ± 0.0443    
            S.GRUBS            0.3736 ± 0.0325      1.0382 ± 0.0662    
            S.GRUBS_GRU       0.4050 ± 0.0309 †    0.9727 ± 0.0486 †   
            S.GRUBS_JTRANS  !0.4208 ± 0.0296 *†  !0.9503 ± 0.0525 *†   
D.NRO_IPA   S.GRU              0.5192 ± 0.0065      0.9775 ± 0.0216    
            S.TRANS            0.5304 ± 0.0080      0.9050 ± 0.0166    
            S.GRUBS            0.5263 ± 0.0068      0.9125 ± 0.0174    
            S.GRUBS_GRU     !0.5395 ± 0.0077 *†   0.8775 ± 0.0165 *†   
            S.GRUBS_JTRANS   0.5385 ± 0.0079 *†  !0.8765 ± 0.0177 *†   
D.NRO_ORTO  S.GRU              0.6941 ± 0.0053      0.6004 ± 0.0130    
            S.TRANS            0.7105 ± 0.0050      0.5636 ± 0.0163    
            S.GRUBS            0.7109 ± 0.0051      0.5531 ± 0.0127    
            S.GRUBS_GRU     !0.7260 ± 0.0041 *†  !0.5237 ± 0.0109 *†   
            S.GRUBS_JTRANS   0.7250 ± 0.0045 *†   0.5246 ± 0.0111 *†   

                                          M.TER                M.FER  \
D.WIKIHAN   S.GRU              0.1724 ± 0.0032      0.0686 ± 0.0026    
            S.TRANS            0.1746 ± 0.0039      0.0696 ± 0.0029    
            S.GRUBS            0.1758 ± 0.0043      0.0701 ± 0.0022    
            S.GRUBS_GRU      0.1650 ± 0.0034 *†   0.0661 ± 0.0018 *†   
            S.GRUBS_JTRANS  !0.1646 ± 0.0038 *†  !0.0658 ± 0.0021 *†   
D.WIKIHAN_A S.GRU              0.1774 ± 0.0030      0.0689 ± 0.0017    
            S.TRANS            0.1714 ± 0.0039      0.0661 ± 0.0020    
            S.GRUBS            0.1690 ± 0.0026      0.0658 ± 0.0014    
            S.GRUBS_GRU     !0.1598 ± 0.0033 *†   0.0628 ± 0.0017 *†   
            S.GRUBS_JTRANS   0.1602 ± 0.0030 *†  !0.0626 ± 0.0017 *†   
D.BAXTER    S.GRU              0.2479 ± 0.0147      0.0914 ± 0.0049    
            S.TRANS            0.2233 ± 0.0108      0.0875 ± 0.0069    
            S.GRUBS            0.2328 ± 0.0148      0.0917 ± 0.0065    
            S.GRUBS_GRU       0.2181 ± 0.0109 †    0.0867 ± 0.0058 †   
            S.GRUBS_JTRANS  !0.2131 ± 0.0118 *†   !0.0850 ± 0.0063 †   
D.NRO_IPA   S.GRU              0.1244 ± 0.0028      0.0390 ± 0.0012    
            S.TRANS            0.1148 ± 0.0018      0.0377 ± 0.0008    
            S.GRUBS           0.1018 ± 0.0019 *    0.0353 ± 0.0009 *   
            S.GRUBS_GRU      0.0979 ± 0.0018 *†   0.0336 ± 0.0007 *†   
            S.GRUBS_JTRANS  !0.0978 ± 0.0020 *†  !0.0333 ± 0.0008 *†   
D.NRO_ORTO  S.GRU              0.0781 ± 0.0018                     -   
            S.TRANS            0.0734 ± 0.0022                     -   
            S.GRUBS           0.0617 ± 0.0014 *                    -   
            S.GRUBS_GRU     !0.0584 ± 0.0012 *†                    -   
            S.GRUBS_JTRANS   0.0585 ± 0.0012 *†                    -   

                                          M.BCF  
D.WIKIHAN   S.GRU              0.7426 ± 0.0038   
            S.TRANS            0.7393 ± 0.0048   
            S.GRUBS            0.7364 ± 0.0064   
            S.GRUBS_GRU      0.7515 ± 0.0048 *†  
            S.GRUBS_JTRANS  !0.7520 ± 0.0052 *

## Correlation

Note results are not cached for correlation test

In [20]:
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

In [25]:
corr_test_res: dict[D, S, dict[M, dict]]
corr_test_res = run_correlation_test(allD, [S.GRUBS_GRU, S.GRUBS_JTRANS])

In [22]:
def format_corr_res(cell):
    if cell['invalidate']:
        return 0
    
    corr_coefficient = cell['corr_coefficient']
    pval = cell['pval']
    return corr_coefficient
    # return f"{corr_coefficient:.4f}; p = {pval:.4f}"

In [23]:
corr_test_df = pd.DataFrame.from_dict(corr_test_res, orient='index')
corr_test_df_html = corr_test_df.copy()
for m in allM:
    corr_test_df_html[m] = corr_test_df_html[m].apply(format_corr_res)
corr_test_df_html

M.ACC     M.TED     M.TER     M.FER     M.BCF
D.WIKIHAN   S.GRUBS_GRU     0.277074  0.364740  0.364739  0.184471  0.363869
            S.GRUBS_JTRANS  0.386046  0.098667  0.098668  0.313243  0.027570
D.WIKIHAN_A S.GRUBS_GRU     0.382992  0.417877  0.417876  0.482918  0.357298
            S.GRUBS_JTRANS  0.184890 -0.043537 -0.043539 -0.065524 -0.031018
D.BAXTER    S.GRUBS_GRU     0.373529  0.123573  0.123572  0.427750  0.017266
            S.GRUBS_JTRANS  0.543157  0.274231  0.274231  0.278224  0.318399
D.NRO_IPA   S.GRUBS_GRU    -0.220702 -0.011459 -0.011457 -0.037341  0.033643
            S.GRUBS_JTRANS -0.070585 -0.018115 -0.018116 -0.086556  0.042051
D.NRO_ORTO  S.GRUBS_GRU     0.253125  0.404426  0.404425  0.000000  0.445893
            S.GRUBS_JTRANS  0.163917  0.112274  0.112273  0.000000  0.103548

In [24]:
corr_test_df_html.style.background_gradient(cmap='bwr', vmin=-2, vmax=2)